# Knowledge Base Integration with Azure AI Foundry Agents

This notebook demonstrates how to integrate **Azure AI Search** (formerly Cognitive Search) as a knowledge base with Azure AI Foundry agents using the **File Search** tool.

## What is Knowledge Base Integration?

Knowledge Base integration enables agents to:
- Search through uploaded documents and files
- Retrieve relevant information using semantic search
- Ground responses in your organization's data
- Provide cited answers from specific documents
- Work with various file formats (PDF, DOCX, TXT, etc.)

## Key Concepts

**Vector Store:**
- Storage for indexed documents
- Supports semantic search with embeddings
- Manages file uploads and indexing
- Can be shared across multiple agents

**File Search Tool:**
- Built-in Azure AI tool for document retrieval
- Automatically chunks and indexes documents
- Uses vector embeddings for semantic search
- Returns relevant passages with citations

**Supported File Types:**
- Text: `.txt`, `.md`, `.csv`
- Documents: `.pdf`, `.docx`, `.doc`
- Code: `.py`, `.js`, `.java`, `.cpp`, etc.
- Data: `.json`, `.xml`, `.yaml`

## Key Benefits

✅ **Private Data**: Search your organization's documents  
✅ **Automatic Indexing**: No manual embedding pipeline needed  
✅ **Semantic Search**: Find relevant content by meaning  
✅ **Citations**: Responses include source references  
✅ **Multi-Format**: Support for various file types  

## Use Cases

- **Internal Knowledge Bases**: Company policies, procedures, documentation
- **Customer Support**: Product manuals, FAQs, troubleshooting guides
- **Research Assistants**: Academic papers, research documents
- **Legal/Compliance**: Contracts, regulations, legal documents
- **Technical Documentation**: API docs, code repositories, wikis

## Prerequisites

Before starting, ensure you have:
1. Azure AI Foundry project created
2. Azure AI Search resource connected to your project
3. Azure credentials configured (Azure CLI or DefaultAzureCredential)
4. Documents to upload (PDF, DOCX, TXT, etc.)

---

## Table of Contents

1. [Part 1: Setup and Configuration](#part-1-setup-and-configuration)
   - Step 1: Import Dependencies and Initialize Client
2. [Part 2: Understanding Knowledge Base Architecture](#part-2-understanding-knowledge-base-architecture)
   - Architecture Overview
   - File Search vs Bing Grounding
3. [Part 3: Create Vector Store](#part-3-create-vector-store)
   - Step 1: Create Vector Store
   - Step 2: Upload Files
   - Step 3: Verify Upload
4. [Part 4: Create Agent with File Search](#part-4-create-agent-with-file-search)
   - Configure File Search Tool
   - Create Knowledge-Enhanced Agent
5. [Part 5: Test the Agent](#part-5-test-the-agent)
   - Query Documents
   - Display Citations
6. [Part 6: Cleanup](#part-6-cleanup)
7. [Summary and Best Practices](#summary-and-best-practices)

---

## Part 1: Setup and Configuration

### Step 1: Import Dependencies and Initialize Client

In [ ]:
import os
import shutil

new_path_entry = "/opt/homebrew/bin"  # Replace with the directory you want to add
current_path = os.environ.get('PATH', '')

if new_path_entry not in current_path.split(os.pathsep):
    os.environ['PATH'] = new_path_entry + os.pathsep + current_path
    print(f"Updated PATH for this session: {os.environ['PATH']}")
else:
    print(f"PATH already contains {new_path_entry}: {current_path}")

# You can then verify with shutil.which again
print(f"Location of 'az' found by kernel now: {shutil.which('az')}")

In [ ]:
import os
import sys
import tempfile
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Load environment variables
load_dotenv()

# Get endpoint from environment
endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")

if not endpoint:
    raise ValueError("Please set AZURE_AI_PROJECT_ENDPOINT in .env")

# Initialize client with Azure credential (following official SDK documentation)
project_client = AIProjectClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential()
)

print("✅ Setup Complete")
print(f"   Project Client: {project_client}")

---

## Part 2: Understanding Knowledge Base Architecture

### Architecture Overview

```
┌─────────────────────────────────────────────────────────────────┐
│                         Your Documents                          │
├─────────────────────────────────────────────────────────────────┤
│  • PDFs          • Word Docs      • Text Files                  │
│  • Markdown      • Code Files     • JSON/XML                    │
└────────────────────┬────────────────────────────────────────────┘
                     │ Upload
                     ▼
┌─────────────────────────────────────────────────────────────────┐
│                       Vector Store                              │
├─────────────────────────────────────────────────────────────────┤
│  • Chunks documents automatically                               │
│  • Creates vector embeddings                                    │
│  • Indexes for semantic search                                  │
│  • Stores metadata and references                               │
└────────────────────┬────────────────────────────────────────────┘
                     │ Attached to
                     ▼
┌─────────────────────────────────────────────────────────────────┐
│                    Azure AI Agent                               │
├─────────────────────────────────────────────────────────────────┤
│  • Receives user query                                          │
│  • Uses File Search tool                                        │
│  • Retrieves relevant passages                                  │
│  • Generates response with citations                            │
└────────────────────┬────────────────────────────────────────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────────────────────┐
│                  Response with Citations                        │
├─────────────────────────────────────────────────────────────────┤
│  • Answer grounded in documents                                 │
│  • Source file references                                       │
│  • Page numbers (for PDFs)                                      │
│  • Relevant passages quoted                                     │
└─────────────────────────────────────────────────────────────────┘
```

### File Search vs Bing Grounding

| Aspect | File Search (Knowledge Base) | Bing Grounding |
|--------|------------------------------|----------------|
| **Data Source** | Your uploaded documents | Public web |
| **Use Case** | Private/internal knowledge | Current events, news |
| **Setup** | Upload files to vector store | Configure Bing connection |
| **Search Type** | Semantic search on embeddings | Web search |
| **Citations** | File name, page number | Website URLs |
| **Update Frequency** | Manual (upload new files) | Real-time (web) |
| **Cost** | Storage + compute | Per API call |
| **Privacy** | Fully private | Public data only |

### When to Use Knowledge Base

**✅ Good Use Cases:**
- Company documentation and policies
- Product manuals and guides
- Research papers and reports
- Legal documents and contracts
- Internal knowledge bases
- Technical documentation

**❌ Not Ideal For:**
- Real-time/current information (use Bing)
- Frequently changing data (requires re-upload)
- Public information (use Bing)
- Structured database queries (use custom tools)

---

## Part 3: Create Vector Store

### Step 1: Create Vector Store

A vector store is a container for your indexed documents.

In [ ]:
# Create a vector store for your documents
vector_store = project_client.agents.vector_stores.create_and_poll(
    name="Company Knowledge Base",
    file_ids=[],  # Start empty, will add files next
    expires_after={
        "anchor": "last_active_at",
        "days": 7  # Auto-delete after 7 days of inactivity
    }
)

print("✅ Vector Store Created")
print(f"   Store ID: {vector_store.id}")
print(f"   Name: {vector_store.name}")
print(f"   Status: {vector_store.status}")
print(f"   File Count: {vector_store.file_counts}")

### Step 2: Upload Files

Upload documents to the vector store. The system will automatically:
- Extract text from documents
- Chunk content into searchable segments
- Generate vector embeddings
- Index for fast retrieval

In [ ]:
# Upload files from the data folder
file_paths = [
    "data/code_of_conduct_temp.txt",
    "data/hr_policies.txt"
]

uploaded_files = []

print("📤 Uploading Files to Vector Store...\n")

for file_path in file_paths:
    if os.path.exists(file_path):
        # Upload file
        with open(file_path, "rb") as f:
            file = project_client.agents.files.upload_and_poll(
                file=f,
                purpose="assistants"
            )

        # Add to vector store
        project_client.agents.vector_store_files.create_and_poll(
            vector_store_id=vector_store.id,
            file_id=file.id
        )

        uploaded_files.append(file)

        print(f"   ✓ {os.path.basename(file_path)}")
        print(f"     File ID: {file.id}")
        # Get file size from the actual file on disk
        file_size = os.path.getsize(file_path)
        print(f"     Size: {file_size} bytes")
        print()
    else:
        print(f"   ⚠️  File not found: {file_path}")

print(f"✅ Uploaded {len(uploaded_files)} file(s) to vector store")

### Step 3: Verify Upload

Check the vector store status and file count.

In [ ]:
# Retrieve updated vector store info
vector_store = project_client.agents.vector_stores.get(vector_store.id)

print("📊 Vector Store Status")
print("=" * 80)
print(f"   Store ID: {vector_store.id}")
print(f"   Name: {vector_store.name}")
print(f"   Status: {vector_store.status}")
print(f"   File Counts: {vector_store.file_counts}")
print(f"   Created At: {vector_store.created_at}")

# List all files in the vector store
files = project_client.agents.vector_store_files.list(vector_store.id)

print(f"\n📁 Files in Vector Store ({len(list(files))} total):")
print("=" * 80)
for file in files:
    print(f"   • File ID: {file.id}")
    print(f"     Status: {file.status}")
    print()

---

## Part 4: Create Agent with File Search

### Configure File Search Tool

Create an agent that can search through the uploaded documents.

In [ ]:
from azure.ai.agents.models import FileSearchTool, ToolResources, FileSearchToolResource

# Create File Search tool configuration
file_search_tool = FileSearchTool(
    vector_store_ids=[vector_store.id]
)

print("✅ File Search Tool Configured")
print(f"   Vector Store ID: {vector_store.id}")
print(f"   Tool Type: {type(file_search_tool).__name__}")

### Create Knowledge-Enhanced Agent

Create an agent with access to your knowledge base.

In [ ]:
# Create agent with file search capability
kb_agent = project_client.agents.create_agent(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o"),
    name="Knowledge Base Assistant",
    instructions="""You are a Knowledge Base Assistant with access to company documents.

## Role
You help employees find information from company policies, procedures, and documentation.

## Guidelines
1. Use the file search tool to find relevant information in the knowledge base
2. Always cite the source document when providing information
3. If information is not found in the documents, clearly state that
4. Provide accurate quotes from the documents when applicable
5. If the question is unclear, ask for clarification
6. Be helpful, accurate, and concise

## Response Format
- Provide clear, direct answers
- Include relevant quotes from documents
- Cite source files and sections
- If multiple documents are relevant, reference all of them""",
    tools=file_search_tool.definitions,
    tool_resources=ToolResources(
        file_search=FileSearchToolResource(
            vector_store_ids=[vector_store.id]
        )
    )
)

print("✅ Knowledge Base Agent Created")
print(f"   Agent ID: {kb_agent.id}")
print(f"   Agent Name: {kb_agent.name}")
print(f"   Tools: File Search (Vector Store: {vector_store.id})")

---

## Part 5: Test the Agent

### Query Documents

Test the agent with queries about the uploaded documents.

In [ ]:
# Create a thread for conversation
kb_thread = project_client.agents.threads.create()

print("💬 Created conversation thread")
print(f"   Thread ID: {kb_thread.id}")

# Test query
test_query = "How many days of annual leave do employees get?"

print(f"\n👤 User Query: {test_query}")
print("\n⏳ Searching knowledge base...\n")

# Create message
message = project_client.agents.messages.create(
    thread_id=kb_thread.id,
    role="user",
    content=test_query
)

# Run agent
run = project_client.agents.runs.create_and_process(
    thread_id=kb_thread.id,
    agent_id=kb_agent.id
)

# Get response
messages = project_client.agents.messages.list(thread_id=kb_thread.id)

print("=" * 80)
print("🤖 Agent Response:")
print("=" * 80)

for msg in messages:
    if msg.role == "assistant":
        for content in msg.content:
            if hasattr(content, 'text'):
                print(content.text.value)

                # Display annotations (citations)
                if hasattr(content.text, 'annotations') and content.text.annotations:
                    print("\n📎 Sources:")
                    for i, annotation in enumerate(content.text.annotations, 1):
                        if hasattr(annotation, 'file_citation'):
                            citation = annotation.file_citation
                            print(f"   [{i}] File ID: {citation.file_id}")
                            if hasattr(citation, 'quote') and citation.quote:
                                quote_preview = citation.quote[:100] if len(
                                    citation.quote) > 100 else citation.quote
                                print(f"       Quote: {quote_preview}...")
        break

print("\n" + "=" * 80)

### Display Citations

Extract and display detailed citation information.

In [ ]:
print("=" * 80)
print("📚 DETAILED CITATIONS")
print("=" * 80)

# Re-fetch messages to ensure we have fresh data
messages_detailed = project_client.agents.messages.list(thread_id=kb_thread.id)

citation_found = False

for msg in messages_detailed:
    if msg.role == "assistant":
        print(f"\n🔍 Examining assistant message...")
        print(f"   Content items: {len(msg.content)}")

        for idx, content in enumerate(msg.content):
            print(f"   Content [{idx}] type: {type(content).__name__}")

            if hasattr(content, 'text'):
                print(f"   Has text attribute: True")

                if hasattr(content.text, 'annotations'):
                    annotations = content.text.annotations
                    print(
                        f"   Annotations count: {len(annotations) if annotations else 0}")

                    if annotations:
                        citation_found = True
                        print(f"\n📌 Found {len(annotations)} citation(s):\n")

                        for i, annotation in enumerate(annotations, 1):
                            print(f"Citation {i}:")
                            print("-" * 60)
                            print(
                                f"   Annotation type: {type(annotation).__name__}")

                            if hasattr(annotation, 'file_citation'):
                                citation = annotation.file_citation
                                print(f"   Has file_citation: True")

                                # Get file details
                                file_id = citation.file_id
                                print(f"   File ID: {file_id}")

                                # Get file info
                                try:
                                    file_info = project_client.agents.files.get(
                                        file_id)
                                    print(
                                        f"   File Name: {file_info.filename}")
                                except Exception as e:
                                    print(f"   File Name: (unavailable - {e})")

                                # Display quote
                                if hasattr(citation, 'quote') and citation.quote:
                                    print(f"\n   Quote:")
                                    print(f"   \"{citation.quote}\"")
                                else:
                                    print(f"\n   Quote: (not available)")

                            print()
                    else:
                        print("   ⚠️ Annotations list is empty")
                else:
                    print("   ⚠️ No annotations attribute")
            else:
                print(f"   No text attribute")
        break

if not citation_found:
    print("\n⚠️ No citations found in any message content")

print("=" * 80)

---

## Part 6: Cleanup

Clean up resources to avoid unnecessary costs.

In [ ]:
# Delete agents
project_client.agents.delete_agent(kb_agent.id)
print(f"✓ Deleted agent: {kb_agent.id}")

# Delete threads
project_client.agents.threads.delete(kb_thread.id)
print(f"✓ Deleted thread: {kb_thread.id}")

# Delete vector stores
project_client.agents.vector_stores.delete(vector_store.id)
print(f"✓ Deleted vector store: {vector_store.id}")

print("\n✅ Cleanup Complete")
print("   • All agents deleted")
print("   • All threads deleted")
print("   • All vector stores deleted")

---

## Summary and Best Practices

### What We Learned

1. **Vector Store Creation**: Container for indexed documents with automatic chunking and embedding
2. **File Upload**: Support for multiple file formats with automatic processing
3. **File Search Tool**: Built-in semantic search over uploaded documents
4. **Agent Integration**: Simple attachment of vector stores to agents
5. **Citation Extraction**: Responses include source references and quotes

### Architecture Summary

```
Documents → Vector Store → File Search Tool → Agent → Cited Responses
```

### Best Practices

#### Document Preparation

✅ **Structure Documents Well**: Use clear headings and sections  
✅ **Keep Files Focused**: One topic per document works best  
✅ **Use Descriptive Names**: Help identify source documents  
✅ **Clean Formatting**: Remove unnecessary formatting  
✅ **Optimal Size**: Documents between 1-100 pages work best  

#### Vector Store Management

✅ **Organize by Domain**: Separate HR, tech docs, legal, etc.  
✅ **Regular Updates**: Keep documents current  
✅ **Set Expiration**: Use auto-delete for temporary stores  
✅ **Monitor Usage**: Track storage and search costs  
✅ **Version Control**: Keep track of document versions  

#### Agent Instructions

✅ **Be Specific**: Tell agent when to use file search  
✅ **Require Citations**: Instruct to always cite sources  
✅ **Handle Missing Info**: Define behavior when info not found  
✅ **Format Responses**: Specify how to present information  
✅ **Clarification**: Encourage asking follow-up questions  

#### Performance Optimization

✅ **Limit Search Results**: Use `max_num_results` parameter  
✅ **Specific Queries**: Encourage users to be specific  
✅ **Cache Common Queries**: Consider caching frequent answers  
✅ **Monitor Latency**: Track search and response times  
✅ **Index Quality**: Regularly review and update documents  

### When to Use File Search

**✅ Ideal For:**
- Internal documentation and policies
- Product manuals and guides
- FAQ databases
- Research paper collections
- Legal/compliance documents
- Technical documentation

**❌ Not Suitable For:**
- Real-time data (use Bing Grounding)
- Structured databases (use custom SQL tools)
- Transactional data (use APIs)
- Frequently changing content (requires re-upload)
- Highly sensitive data (consider security implications)

### Security Considerations

✅ **Access Control**: Limit who can upload/access documents  
✅ **Data Classification**: Review documents for sensitivity  
✅ **Audit Logs**: Track document access and queries  
✅ **Encryption**: Data encrypted at rest and in transit  
✅ **Retention**: Set appropriate expiration policies  

### Next Steps

1. **Upload Your Documents**: Start with a small set of documents
2. **Test Queries**: Validate search quality with real questions
3. **Optimize Instructions**: Refine agent instructions based on results
4. **Scale Up**: Add more documents and vector stores
5. **Monitor Performance**: Track metrics and optimize
6. **Combine Tools**: Integrate with Bing, custom functions

### Additional Resources

- [File Search Documentation](https://learn.microsoft.com/azure/ai-services/openai/how-to/file-search)
- [Vector Store API](https://learn.microsoft.com/azure/ai-studio/how-to/vector-stores)
- [Supported File Types](https://learn.microsoft.com/azure/ai-studio/concepts/retrieval-augmented-generation)
- [Best Practices for RAG](https://learn.microsoft.com/azure/ai-studio/concepts/rag-best-practices)
- [Azure AI Search](https://learn.microsoft.com/azure/search/)